# Deciphering Code with Character-Level RNN





## Dataset

In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv('../data/Caesar.csv')
df

,Passwords,ciphertext,key
0,2428031609,3539142710,19
1,4531040045,0197606601,4
2,almigrana1,sdeayjsfs3,8
3,quiero95,nrfbol62,3
4,doitnow2,vgalfgo4,8
...,...,...,...
199995,dime168,otxp613,15
199996,!l0v3k3v!n,x6h9w9hz,14
199997,9870tmf05,3214ngz49,6
199998,CAMILA,IGSORG,20


In [3]:
for i in range(len(df['Passwords'])):
    if len(df['Passwords'][i]) > 20:
        df.drop(i, axis=0, inplace=True)

In [4]:
df = df.dropna()[:120000].reset_index(drop=True)

In [5]:
df['Passwords'].apply(str)
df['ciphertext'].apply(str)
df

,Passwords,ciphertext,key
0,2428031609,3539142710,19
1,4531040045,0197606601,4
2,almigrana1,sdeayjsfs3,8
3,quiero95,nrfbol62,3
4,doitnow2,vgalfgo4,8
...,...,...,...
119995,studme,uvwfog,24
119996,rockyou816,vsgocsy816,22
119997,redred2976,dqpdqp8532,14
119998,knoxx,ehirr,6


## Preprocessing Data

In [6]:
def tokenize(x):
    x_tk = Tokenizer(char_level=True)
    x_tk.fit_on_texts(x)                 
    return x_tk.texts_to_sequences(x), x_tk

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding="post", truncating="post",)

### Preprocess Pipeline

In [7]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [8]:
preproc_code_sentences, preproc_plaintext_sentences, code_tokenizer, plaintext_tokenizer = preprocess(df['Passwords'], df['ciphertext'])

In [9]:
preproc_code_sentences[0]

array([ 7, 17,  7, 15,  5, 13,  3, 19,  5, 12,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])

In [10]:
len(code_tokenizer.word_index)+1

121

In [11]:
len(plaintext_tokenizer.word_index)+1

51

In [12]:
plaintext_tokenizer.word_index

{'1': 1,
 '0': 2,
 '2': 3,
 '3': 4,
 '9': 5,
 '8': 6,
 '5': 7,
 '4': 8,
 '7': 9,
 '6': 10,
 'q': 11,
 'x': 12,
 'j': 13,
 'z': 14,
 'f': 15,
 'w': 16,
 'v': 17,
 'c': 18,
 'g': 19,
 'd': 20,
 'u': 21,
 'k': 22,
 'p': 23,
 'b': 24,
 'h': 25,
 'm': 26,
 'y': 27,
 't': 28,
 'n': 29,
 's': 30,
 'r': 31,
 'o': 32,
 'l': 33,
 'i': 34,
 'e': 35,
 'a': 36,
 'à': 37,
 'ã': 38,
 '¼': 39,
 'â': 40,
 'ä': 41,
 '³': 42,
 '¹': 43,
 'î': 44,
 'ï': 45,
 '²': 46,
 'ª': 47,
 'å': 48,
 'ö': 49,
 'á': 50}

In [13]:
preproc_code_sentences.shape

(120000, 20)

In [14]:
preproc_plaintext_sentences.shape

(120000, 20, 1)

# LSTM

In [15]:
from keras.layers import GRU, Input, Dense, TimeDistributed, LSTM
from keras.models import Model, Sequential
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow


def lstm(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = LSTM(units= 128, return_sequences = True, name='Layer1')(x)
    seq = LSTM(units= 128, return_sequences = True, name='Layer2')(seq)
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer3'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss=sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

tmp_x = pad(preproc_code_sentences, preproc_plaintext_sentences.shape[1]) 
tmp_x = tmp_x.reshape((-1, preproc_plaintext_sentences.shape[-2], 1))     

In [16]:
tmp_x.shape

(120000, 20, 1)

In [17]:
lstm_model = lstm(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
Layer1 (LSTM)                (None, 20, 128)           66560     
_________________________________________________________________
Layer2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 51)            6579      
Total params: 204,723
Trainable params: 204,723
Non-trainable params: 0
_________________________________________________________________


In [18]:
lstm_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=512, epochs=15, validation_split=0.3)

Epoch 1/15
165/165 [==============================] - 20s 23ms/step - loss: 2.0352 - accuracy: 0.5530 - val_loss: 1.5529 - val_accuracy: 0.5847
Epoch 2/15
165/165 [==============================] - 3s 16ms/step - loss: 1.5276 - accuracy: 0.5869 - val_loss: 1.4815 - val_accuracy: 0.5896
Epoch 3/15
165/165 [==============================] - 3s 16ms/step - loss: 1.4665 - accuracy: 0.5934 - val_loss: 1.4481 - val_accuracy: 0.5934
Epoch 4/15
165/165 [==============================] - 3s 16ms/step - loss: 1.4397 - accuracy: 0.5952 - val_loss: 1.4382 - val_accuracy: 0.5966
Epoch 5/15
165/165 [==============================] - 3s 16ms/step - loss: 1.4208 - accuracy: 0.5983 - val_loss: 1.4160 - val_accuracy: 0.5960
Epoch 6/15
165/165 [==============================] - 3s 16ms/step - loss: 1.4079 - accuracy: 0.5999 - val_loss: 1.4080 - val_accuracy: 0.5989
Epoch 7/15
165/165 [==============================] - 3s 16ms/step - loss: 1.4016 - accuracy: 0.6014 - val_loss: 1.3981 - val_accuracy: 0.601

In [19]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''
    return ''.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

def getPred(model, x, n):
    print('Predicted\tActual')
    for i in range(n):
        print(logits_to_text(model.predict([x[:n]])[i], plaintext_tokenizer), end='\t')
        print(df['Passwords'][i])

In [20]:
getPred(lstm_model, tmp_x, 5)

Predicted	Actual
0623031609	2428031609
5631050056	4531040045
tivvsasvs1	almigrana1
sssav296	quiero95
gs0922s2	doitnow2


# GRU

In [21]:
def gru(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = GRU(units= 128, return_sequences = True, name='Layer1')(x)
    seq = GRU(units= 128, return_sequences = True, name='Layer2')(seq)
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer3'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

In [22]:
gru_model = gru(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
Layer1 (GRU)                 (None, 20, 128)           50304     
_________________________________________________________________
Layer2 (GRU)                 (None, 20, 128)           99072     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 51)            6579      
Total params: 155,955
Trainable params: 155,955
Non-trainable params: 0
_________________________________________________________________


In [23]:
gru_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=512, epochs=15, validation_split=0.3)

Epoch 1/15
165/165 [==============================] - 7s 20ms/step - loss: 2.0319 - accuracy: 0.5571 - val_loss: 1.5188 - val_accuracy: 0.5850
Epoch 2/15
165/165 [==============================] - 2s 14ms/step - loss: 1.4967 - accuracy: 0.5879 - val_loss: 1.4562 - val_accuracy: 0.5912
Epoch 3/15
165/165 [==============================] - 2s 14ms/step - loss: 1.4398 - accuracy: 0.5938 - val_loss: 1.4228 - val_accuracy: 0.5936
Epoch 4/15
165/165 [==============================] - 2s 14ms/step - loss: 1.4120 - accuracy: 0.5972 - val_loss: 1.4016 - val_accuracy: 0.5978
Epoch 5/15
165/165 [==============================] - 2s 14ms/step - loss: 1.3886 - accuracy: 0.6009 - val_loss: 1.3865 - val_accuracy: 0.6015
Epoch 6/15
165/165 [==============================] - 2s 14ms/step - loss: 1.3760 - accuracy: 0.6026 - val_loss: 1.3740 - val_accuracy: 0.6008
Epoch 7/15
165/165 [==============================] - 2s 14ms/step - loss: 1.3629 - accuracy: 0.6045 - val_loss: 1.3606 - val_accuracy: 0.6027

In [24]:
getPred(gru_model, tmp_x, 5)

Predicted	Actual
0828031509	2428031609
n531030035	4531040045
ozznnzzut1	almigrana1
uznzuu94	quiero95
nuuuzuu2	doitnow2
